# Deep Q-Learning 

For this assignment we will implement the Deep Q-Learning algorithm with Experience Replay as described in breakthrough paper __"Playing Atari with Deep Reinforcement Learning"__. We will train an agent to play the famous game of __Breakout__.

In [1]:
import sys
import gym
import torch
import pylab
import random
import numpy as np
import time
from collections import deque
from datetime import datetime
from copy import deepcopy
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
torch.backends.cudnn.benchmarks = True
from torch.autograd import Variable
from utils import *
from agent import *
from model import *
from config import *
from env import GameEnv
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Understanding the environment

In the following cell, we initialise our game of __Breakout__ and you can see how the environment looks like. For further documentation of the of the environment refer to https://gym.openai.com/envs. 

In [2]:
envs = []
for i in range(num_envs):
    envs.append(GameEnv('SpaceInvadersDeterministic-v4'))
#env.render()

/home/sigai/anaconda3/envs/py36/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [3]:
number_lives = envs[0].life
state_size = envs[0].observation_space.shape
action_size = envs[0].action_space.n
rewards, episodes = [], []

## Creating a DQN Agent

Here we create a DQN Agent. This agent is defined in the __agent.py__. The corresponding neural network is defined in the __model.py__. 

__Evaluation Reward__ : The average reward received in the past 100 episodes/games.

__Frame__ : Number of frames processed in total.

__Memory Size__ : The current size of the replay memory.

### Main Training Loop

In [ ]:
agent = Agent(action_size, mode='PPO_MHDPA')
torch.save(agent.policy_net.state_dict(), "./save_model/spaceinvaders_ppo_best")
evaluation_reward = deque(maxlen=evaluation_reward_length)
frame = 0
memory_size = 0
reset_max = 10


### Loop through all environments and run PPO on them

#env_names = ['Breakout-v0', 'Phoenix-v0', 'Asteroids-v0', 'SpaceInvaders-v0', 'MsPacman-v0', 'Asterix-v0', 'Atlantis-v0', 'Alien-v0', 'Amidar-v0', 'Assault-v0', 'BankHeist-v0']
env_names = ['SpaceInvaders-v4']
for a in range(len(env_names)):
    name = env_names[a]
    print("\n\n\n ------- STARTING TRAINING FOR %s ------- \n\n\n" % (name))
    
    envs = []
    for i in range(num_envs):
        envs.append(GameEnv(name))
    #env.render()
    

    number_lives = envs[0].life
    state_size = envs[0].observation_space.shape
    if (name == 'SpaceInvaders-v0' or name == 'Breakout-v0'):
        action_size = 4
    else:
        action_size = envs[0].action_space.n
    rewards, episodes = [], []

    vis_env_idx = 0
    vis_env = envs[vis_env_idx]
    e = 0
    frame = 0
    max_eval = -np.inf
    reset_count = 0


    agent = Agent(action_size, mode='PPO_MHDPA')
    torch.save(agent.policy_net.state_dict(), "./save_model/" + name + "_best")
    evaluation_reward = deque(maxlen=evaluation_reward_length)
    frame = 0
    memory_size = 0
    reset_max = 10
    
    print("Determing min/max rewards of environment")
    [low, high] = score_range = get_score_range(name)
    print("Min: %d. Max: %d." % (low, high))

    while (frame < 10000000):
        step = 0
        assert(num_envs * env_mem_size == train_frame)
        frame_next_vals = []
        
        for j in range(env_mem_size):
            
            curr_states = np.stack([envs[i].history[HISTORY_SIZE-1,:,:] for i in range(num_envs)])
            next_states = []
            net_in = np.stack([envs[i].history[:HISTORY_SIZE,:,:] for i in range(num_envs)])
            step += num_envs
            frame += num_envs
            actions, values, _ = agent.get_action(np.float32(net_in) / 255.)
            
            for i in range(num_envs):
                env = envs[i]
                next_state, env.reward, env.done, env.info = env.step(actions[i])
                next_states.append(next_state)
                if (i == vis_env_idx):
                    vis_env._env.render()
            
            for i in range(num_envs):
                env = envs[i]
                """
                next_state, env.reward, env.done, env.info = env.step(actions[i])
                if (i == vis_env_idx):
                    vis_env._env.render()
                """
                
                frame_next_state = get_frame(next_states[i])
                env.history[HISTORY_SIZE,:,:] = frame_next_state
                terminal_state = check_live(env.life, env.info['ale.lives'])
                env.life = env.info['ale.lives']
                r = (env.reward / high) * 20.0 #np.log(max(env.reward+1, 1))#((env.reward - low) / (high - low)) * 30
                agent.memory.push(i, deepcopy(curr_states[i]), actions[i], r, terminal_state, values[i], 0, 0)
                
                if (j == env_mem_size-1):
                    net_in = np.stack([envs[k].history[1:,:,:] for k in range(num_envs)])
                    _, frame_next_vals, _ = agent.get_action(np.float32(net_in) / 255.)
                
                env.score += env.reward
                env.history[:HISTORY_SIZE, :, :] = env.history[1:,:,:]
        
                if (env.done):
                    if (e % 50 == 0):
                        print('now time : ', datetime.now())
                        rewards.append(np.mean(evaluation_reward))
                        episodes.append(e)
                        pylab.plot(episodes, rewards, 'b')
                        pylab.savefig("./save_graph/" + name + "_ppo.png")
                        torch.save(agent.policy_net, "./save_model/" + name + "_ppo")

                        if np.mean(evaluation_reward) > max_eval:
                            torch.save(agent.policy_net.state_dict(), "./save_model/"  + name + "_ppo_best")
                            max_eval = float(np.mean(evaluation_reward))
                            reset_count = 0
                        elif e > 5000:
                            reset_count += 1
                            """
                            if (reset_count == reset_max):
                                print("Training went nowhere, starting again at best model")
                                agent.policy_net.load_state_dict(torch.load("./save_model/spaceinvaders_ppo_best"))
                                agent.update_target_net()
                                reset_count = 0
                            """
                    e += 1
                    evaluation_reward.append(env.score)
                    print("episode:", e, "  score:", env.score,  " epsilon:", agent.epsilon, "   steps:", step,
                      " evaluation reward:", np.mean(evaluation_reward))

                    env.done = False
                    env.score = 0
                    env.history = np.zeros([HISTORY_SIZE+1,84,84], dtype=np.uint8)
                    env.state = env.reset()
                    env.life = number_lives
                    get_init_state(env.history, env.state)
            
        agent.train_policy_net(frame, frame_next_vals)
        agent.update_target_net()
    print("FINISHED TRAINING FOR %s" % (name))
    pylab.figure()
    
    for i in range(len(envs)):
        envs[i]._env.close()
    del envs

In [ ]:
def test_best(name):
    env = GameEnv(name)
    print("\n\n\n ------- TESTING BEST MODEL FOR %s ------- \n\n\n" % (name))
    number_lives = env.life
    
    if (name == 'SpaceInvaders-v0'):
        action_size = 4
    else:
        action_size = env.action_space.n
    rewards, episodes = [], []
    
    e = 0
    frame = 0

    agent = Agent(action_size)
    agent.policy_net.load_state_dict(torch.load("./save_model/" + name + "_ppo_best"))
    agent.update_target_net()
    agent.policy_net.eval()
    evaluation_reward = deque(maxlen=evaluation_reward_length)

    for i in range(100):
        env.done = False
        env.score = 0
        env.history = np.zeros([HISTORY_SIZE+1,84,84], dtype=np.uint8)
        env.state = env.reset()
        env.life = number_lives
        get_init_state(env.history, env.state)
        step = 0
        while not env.done:
            curr_state = env.history[HISTORY_SIZE-1,:,:]
            net_in = env.history[:HISTORY_SIZE,:,:]
            action, value, _ = agent.get_action(np.float32(net_in) / 255.)
            
            next_state, env.reward, env.done, env.info = env.step(action)
            env._env.render()
            
            frame_next_state = get_frame(next_state)
            
            env.history[HISTORY_SIZE,:,:] = frame_next_state
            terminal_state = check_live(env.life, env.info['ale.lives'])
            env.life = env.info['ale.lives']
            
            
            env.score += env.reward
            env.history[:HISTORY_SIZE, :, :] = env.history[1:,:,:]
            step += 1
        

        evaluation_reward.append(env.score)
        print("episode:", e, "  score:", env.score,  " epsilon:", agent.epsilon, "   steps:", step,
                      " evaluation reward:", np.mean(evaluation_reward))
            

In [ ]:
test_best('MsPacman-v0')

### Convolutional LSTM agent

In [ ]:
agent = Agent(action_size, mode='PPO_LSTM')
torch.save(agent.policy_net.state_dict(), "./save_model/spaceinvaders_ppo_best")
evaluation_reward = deque(maxlen=evaluation_reward_length)
frame = 0
memory_size = 0
reset_max = 10


### Loop through all environments and run PPO on them

#env_names = ['Breakout-v0', 'Phoenix-v0', 'Asteroids-v0', 'SpaceInvaders-v0', 'MsPacman-v0', 'Asterix-v0', 'Atlantis-v0', 'Alien-v0', 'Amidar-v0', 'Assault-v0', 'BankHeist-v0']
env_names = ['SpaceInvaders-v4']
for a in range(len(env_names)):
    name = env_names[a]
    print("\n\n\n ------- STARTING TRAINING FOR %s ------- \n\n\n" % (name))
    
    envs = []
    for i in range(num_envs):
        envs.append(GameEnv(name))
        envs[i].reset_memory(agent.init_hidden())
    #env.render()
    

    number_lives = envs[0].life
    state_size = envs[0].observation_space.shape
    if (name == 'SpaceInvaders-v0' or name == 'Breakout-v0'):
        action_size = 4
    else:
        action_size = envs[0].action_space.n
    rewards, episodes = [], []

    vis_env_idx = 0
    vis_env = envs[vis_env_idx]
    e = 0
    frame = 0
    max_eval = -np.inf
    reset_count = 0


    agent = Agent(action_size, mode='PPO_LSTM')
    torch.save(agent.policy_net.state_dict(), "./save_model/" + name + "_best")
    evaluation_reward = deque(maxlen=evaluation_reward_length)
    frame = 0
    memory_size = 0
    reset_max = 10
    
    print("Determing min/max rewards of environment")
    [low, high] = score_range = get_score_range(name)
    print("Min: %d. Max: %d." % (low, high))

    while (frame < 10000000):
        step = 0
        assert(num_envs * env_mem_size == train_frame)
        frame_next_vals = []
        
        for j in range(env_mem_size):
            
            curr_states = np.stack([envs[i].history[[HISTORY_SIZE-1],:,:] for i in range(num_envs)])
            hiddens = torch.cat([envs[i].memory for i in range(num_envs)])
            next_states = []
            step += num_envs
            frame += num_envs
            actions, values, hiddens = agent.get_action(np.float32(curr_states) / 255., hiddens)
            hiddens = hiddens.detach()
            
            for i in range(num_envs):
                env = envs[i]
                next_state, env.reward, env.done, env.info = env.step(actions[i])
                next_states.append(next_state)
                if (i == vis_env_idx):
                    vis_env._env.render()
            
            for i in range(num_envs):
                env = envs[i]
                """
                next_state, env.reward, env.done, env.info = env.step(actions[i])
                if (i == vis_env_idx):
                    vis_env._env.render()
                """
                
                frame_next_state = get_frame(next_states[i])
                env.history[HISTORY_SIZE,:,:] = frame_next_state
                env.memory = hiddens[[i]]
                terminal_state = check_live(env.life, env.info['ale.lives'])
                env.life = env.info['ale.lives']
                r = (env.reward / high) * 20.0 #np.log(max(env.reward+1, 1))#((env.reward - low) / (high - low)) * 30
                agent.memory.push(i, [deepcopy(curr_states[i]), hiddens[i].detach().cpu().data.numpy()], actions[i], r, terminal_state, values[i], 0, 0)
                
                if (j == env_mem_size-1):
                    #net_in = np.stack([envs[k].history[1:,:,:] for k in range(num_envs)])
                    net_in = np.stack([envs[k].history[[-1],:,:] for k in range(num_envs)])
                    _, frame_next_vals, _ = agent.get_action(np.float32(net_in) / 255., hiddens)
                
                env.score += env.reward
                env.history[:HISTORY_SIZE, :, :] = env.history[1:,:,:]
        
                if (env.done):
                    if (e % 50 == 0):
                        print('now time : ', datetime.now())
                        rewards.append(np.mean(evaluation_reward))
                        episodes.append(e)
                        pylab.plot(episodes, rewards, 'b')
                        pylab.savefig("./save_graph/" + name + "_ppo.png")
                        torch.save(agent.policy_net, "./save_model/" + name + "_ppo")

                        if np.mean(evaluation_reward) > max_eval:
                            torch.save(agent.policy_net.state_dict(), "./save_model/"  + name + "_ppo_best")
                            max_eval = float(np.mean(evaluation_reward))
                            reset_count = 0
                        elif e > 5000:
                            reset_count += 1
                            """
                            if (reset_count == reset_max):
                                print("Training went nowhere, starting again at best model")
                                agent.policy_net.load_state_dict(torch.load("./save_model/spaceinvaders_ppo_best"))
                                agent.update_target_net()
                                reset_count = 0
                            """
                    e += 1
                    evaluation_reward.append(env.score)
                    print("episode:", e, "  score:", env.score,  " epsilon:", agent.epsilon, "   steps:", step,
                      " evaluation reward:", np.mean(evaluation_reward))

                    env.done = False
                    env.score = 0
                    env.history = np.zeros([HISTORY_SIZE+1,84,84], dtype=np.uint8)
                    env.state = env.reset()
                    env.life = number_lives
                    get_init_state(env.history, env.state)
                    env.reset_memory(agent.init_hidden())
            
        agent.train_policy_net(frame, frame_next_vals)
        agent.update_target_net()
    print("FINISHED TRAINING FOR %s" % (name))
    pylab.figure()
    
    for i in range(len(envs)):
        envs[i]._env.close()
    del envs




 ------- STARTING TRAINING FOR SpaceInvaders-v4 ------- 



Determing min/max rewards of environment
Min: 0. Max: 200.


/home/sigai/anaconda3/envs/py36/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/sigai/anaconda3/envs/py36/lib/python3.6/site-packages/torch/nn/functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/model.py:124: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(x[:,:self.action_size] - torch.max(x[:,:self.action_size],1)[0].unsqueeze(1))


Training network. lr: 0.000250. clip: 0.100000


/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/model.py:140: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(x[:,:self.action_size] - torch.max(x[:,:self.action_size],1)[0].unsqueeze(1))
/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/agent.py:284: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  pol_loss += pol_avg.detach().cpu()[0]
/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/agent.py:285: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  vf_loss += value_loss.detach().cpu()[0]
/home/sigai/Documents/Projects/Object-Recognition/assignment5_m

Iteration 1: Policy loss: -0.003072. Value loss: 0.022652. Entropy: 0.299232.
Iteration 2: Policy loss: -0.003580. Value loss: 0.004498. Entropy: 0.298123.
Iteration 3: Policy loss: -0.001997. Value loss: 0.003115. Entropy: 0.299286.
Training network. lr: 0.000250. clip: 0.100000
Iteration 4: Policy loss: -0.260096. Value loss: 0.115091. Entropy: 0.296358.
Iteration 5: Policy loss: -0.265775. Value loss: 0.115090. Entropy: 0.299206.
Iteration 6: Policy loss: -0.264243. Value loss: 0.107152. Entropy: 0.296674.
now time :  2019-09-05 14:14:58.883665
episode: 1   score: 30.0  epsilon: 1.0    steps: 1016  evaluation reward: 30.0
Training network. lr: 0.000250. clip: 0.100000


...

Iteration 24414: Policy loss: 0.090017. Value loss: 0.035248. Entropy: 0.279645.
Training network. lr: 0.000063. clip: 0.025046
Iteration 24415: Policy loss: 0.014776. Value loss: 0.128832. Entropy: 0.308530.
Iteration 24416: Policy loss: 0.010595. Value loss: 0.057898. Entropy: 0.308714.
Iteration 24417: Policy loss: 0.000551. Value loss: 0.040271. Entropy: 0.308631.
episode: 8270   score: 390.0  epsilon: 1.0    steps: 1008  evaluation reward: 498.9
Training network. lr: 0.000063. clip: 0.025046
Iteration 24418: Policy loss: -0.090048. Value loss: 0.119803. Entropy: 0.311917.
Iteration 24419: Policy loss: -0.093670. Value loss: 0.065531. Entropy: 0.311346.
Iteration 24420: Policy loss: -0.096585. Value loss: 0.050746. Entropy: 0.311243.
episode: 8271   score: 275.0  epsilon: 1.0    steps: 160  evaluation reward: 497.45
Training network. lr: 0.000063. clip: 0.025046
Iteration 24421: Policy loss: 0.146955. Value loss: 0.045665. Entropy: 0.283619.
Iteration 24422: Policy loss: 0.143858. 

episode: 8287   score: 580.0  epsilon: 1.0    steps: 376  evaluation reward: 496.85
episode: 8288   score: 335.0  epsilon: 1.0    steps: 672  evaluation reward: 496.85
episode: 8289   score: 400.0  epsilon: 1.0    steps: 896  evaluation reward: 498.0
Training network. lr: 0.000062. clip: 0.024899
Iteration 24484: Policy loss: -0.347422. Value loss: 0.351498. Entropy: 0.270661.
Iteration 24485: Policy loss: -0.339318. Value loss: 0.154337. Entropy: 0.270828.
Iteration 24486: Policy loss: -0.339086. Value loss: 0.112843. Entropy: 0.271011.
episode: 8290   score: 720.0  epsilon: 1.0    steps: 936  evaluation reward: 498.8
Training network. lr: 0.000062. clip: 0.024899
Iteration 24487: Policy loss: 0.186204. Value loss: 0.154071. Entropy: 0.304478.
Iteration 24488: Policy loss: 0.190796. Value loss: 0.072432. Entropy: 0.303174.
Iteration 24489: Policy loss: 0.185023. Value loss: 0.048480. Entropy: 0.302861.
Training network. lr: 0.000062. clip: 0.024899
Iteration 24490: Policy loss: 0.0303

Training network. lr: 0.000062. clip: 0.024742
Iteration 24550: Policy loss: 0.209590. Value loss: 0.424789. Entropy: 0.297083.
Iteration 24551: Policy loss: 0.214206. Value loss: 0.202050. Entropy: 0.297104.
Iteration 24552: Policy loss: 0.211050. Value loss: 0.098946. Entropy: 0.296472.
Training network. lr: 0.000061. clip: 0.024585
Iteration 24553: Policy loss: 0.080842. Value loss: 0.148883. Entropy: 0.303897.
Iteration 24554: Policy loss: 0.079248. Value loss: 0.078777. Entropy: 0.303485.
Iteration 24555: Policy loss: 0.075857. Value loss: 0.057344. Entropy: 0.304401.
episode: 8308   score: 530.0  epsilon: 1.0    steps: 536  evaluation reward: 499.6
episode: 8309   score: 270.0  epsilon: 1.0    steps: 632  evaluation reward: 500.2
Training network. lr: 0.000061. clip: 0.024585
Iteration 24556: Policy loss: 0.080488. Value loss: 0.148854. Entropy: 0.273447.
Iteration 24557: Policy loss: 0.086128. Value loss: 0.069177. Entropy: 0.273901.
Iteration 24558: Policy loss: 0.092705. Value

Iteration 24617: Policy loss: -0.191766. Value loss: 0.140903. Entropy: 0.306027.
Iteration 24618: Policy loss: -0.202964. Value loss: 0.085269. Entropy: 0.305784.
episode: 8328   score: 565.0  epsilon: 1.0    steps: 104  evaluation reward: 482.2
episode: 8329   score: 595.0  epsilon: 1.0    steps: 872  evaluation reward: 483.75
Training network. lr: 0.000061. clip: 0.024438
Iteration 24619: Policy loss: 0.141199. Value loss: 0.152142. Entropy: 0.300291.
Iteration 24620: Policy loss: 0.146375. Value loss: 0.053792. Entropy: 0.299839.
Iteration 24621: Policy loss: 0.141768. Value loss: 0.035099. Entropy: 0.299343.
Training network. lr: 0.000061. clip: 0.024438
Iteration 24622: Policy loss: 0.001728. Value loss: 0.090001. Entropy: 0.309952.
Iteration 24623: Policy loss: 0.006513. Value loss: 0.048013. Entropy: 0.309868.
Iteration 24624: Policy loss: 0.007602. Value loss: 0.036674. Entropy: 0.309985.
episode: 8330   score: 650.0  epsilon: 1.0    steps: 712  evaluation reward: 483.0
episod

Iteration 24684: Policy loss: 0.116256. Value loss: 0.053527. Entropy: 0.293941.
episode: 8349   score: 545.0  epsilon: 1.0    steps: 208  evaluation reward: 487.55
Training network. lr: 0.000061. clip: 0.024281
Iteration 24685: Policy loss: -0.166642. Value loss: 0.281432. Entropy: 0.295353.
Iteration 24686: Policy loss: -0.157152. Value loss: 0.122279. Entropy: 0.295162.
Iteration 24687: Policy loss: -0.168361. Value loss: 0.079756. Entropy: 0.296232.
Training network. lr: 0.000061. clip: 0.024281
Iteration 24688: Policy loss: -0.001016. Value loss: 0.138601. Entropy: 0.303282.
Iteration 24689: Policy loss: 0.008094. Value loss: 0.066677. Entropy: 0.304028.
Iteration 24690: Policy loss: 0.008127. Value loss: 0.044577. Entropy: 0.302488.
Training network. lr: 0.000061. clip: 0.024281
Iteration 24691: Policy loss: 0.007825. Value loss: 0.281477. Entropy: 0.305760.
Iteration 24692: Policy loss: -0.004129. Value loss: 0.125106. Entropy: 0.305885.
Iteration 24693: Policy loss: 0.002370. V